In [ ]:
import collections

In [ ]:
def read_map(filename):
    """Store the map of caves like a bi-directional graph."""
    
    caves = collections.defaultdict(list)
    with open(filename) as file:
        for line in file:
            start, end = line.strip().split("-")
            caves[start].append(end)
            caves[end].append(start)
    return caves

In [ ]:
caves = read_map("day12.input")

# Part 1

Search recursively through all possibilites, and store the solution in a common list if/when we find the `end` node.

Notes:

* We need to make a `.copy()` of the visited-list, so that each branch of the recursion can keep track of it's own history.
* It would have been better to use a `set` to store the visited caves (faster to check for `in visited`), but sets are not ordered. We could have used a dictionary, but it would have made the code less readable.
* We could have used just a counter for the number of solutions, but it's nice to be able to see the actual paths :)

In [ ]:
def find_paths(cave, visited=None, solutions=None):

    # Initialization at first function-call
    if visited is None:
        visited = []
    if solutions is None:
        solutions = []

    visited.append(cave)
    if cave == "end":
        solutions.append(visited)
    else:
        for exit in caves[cave]:
            if (exit.islower() and (exit not in visited)) or exit.isupper():
                find_paths(exit, visited.copy(), solutions)

    return solutions

In [ ]:
solutions = find_paths("start")
len(solutions)

# Part 2

We add a new variable that keeps track of whether we have already visited a lowercase cave twice.

We can now enter a cave if:
* It is uppercase
* It is lowercase AND ((has not been visited before) OR (no lowercase caves has been visited twice)).

In addition, we need a special check to ensure we don't visit `start` more than once (since it is lowercase).

In [ ]:
def find_paths(cave, visited=None, solutions=None, lower_visited_twice=False):

    # Initialization at first function-call
    if visited is None:
        visited = []
    if solutions is None:
        solutions = []

    if cave.islower() and (cave in visited):
        lower_visited_twice = True
    visited.append(cave)

    if cave == "end":
        solutions.append(visited)
    else:
        for exit in caves[cave]:
            if exit == "start":
                continue
            if (exit.islower() and ((exit not in visited) or not lower_visited_twice)) or exit.isupper():
                find_paths(exit, visited.copy(), solutions, lower_visited_twice)

    return solutions

In [ ]:
solutions = find_paths("start")
len(solutions)

## Faster solution

Here's an example of a faster solution that uses a `set()` to keep track of visited caves, and just calculates the *number of* possible paths. The downside is that it doesn't tell you what the actual paths are (if you're interested).

In [ ]:
def find_paths_faster(cave, visited=None, lower_visited_twice=False):

    solutions = 0
    if visited is None:
        visited = set()
    
    if cave.islower() and (cave in visited):
        lower_visited_twice = True
    visited.add(cave)
    
    if cave == "end":
        return 1
    else:
        for exit in caves[cave]:
            if exit == "start":
                continue
            if (exit.islower() and ((exit not in visited) or not lower_visited_twice)) or exit.isupper():
                solutions += find_paths_faster(exit, visited.copy(), lower_visited_twice)
    
    return solutions

In [ ]:
find_paths_faster("start")